In [1]:
!mkdir ade20k
!wget http://data.csail.mit.edu/places/ADEchallenge/ADEChallengeData2016.zip -P ade20k

--2022-08-16 12:27:27--  http://data.csail.mit.edu/places/ADEchallenge/ADEChallengeData2016.zip
Resolving data.csail.mit.edu (data.csail.mit.edu)... 128.52.129.40
Connecting to data.csail.mit.edu (data.csail.mit.edu)|128.52.129.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 967382037 (923M) [application/zip]
Saving to: ‘ade20k/ADEChallengeData2016.zip’

ADEChallengeData201 100%[===================>] 922.57M  88.7MB/s    in 11s     

2022-08-16 12:27:38 (84.5 MB/s) - ‘ade20k/ADEChallengeData2016.zip’ saved [967382037/967382037]



In [2]:
#!wget http://data.csail.mit.edu/places/ADEchallenge/release_test.zip -P ade20k

In [3]:
!pip3 install openmim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 3.2 MB/s 
     |████████████████████████████████| 235 kB 10.1 MB/s 
     |████████████████████████████████| 51 kB 8.9 MB/s 


In [4]:
!mim install mmcv-full

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 40.6 MB 8.7 MB/s 
     |████████████████████████████████| 190 kB 5.6 MB/s 


In [5]:
!git clone https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!pip install -e .

Cloning into 'mmsegmentation'...
remote: Enumerating objects: 10481, done.
remote: Counting objects: 100% (356/356), done.
remote: Compressing objects: 100% (234/234), done.
remote: Total 10481 (delta 160), reused 280 (delta 120), pack-reused 10125
Receiving objects: 100% (10481/10481), 14.16 MiB | 27.57 MiB/s, done.
Resolving deltas: 100% (7609/7609), done.
/content/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmsegmentation
     |████████████████████████████████| 578 kB 5.2 MB/s 
  Running setup.py develop for mmsegmentation


In [6]:
!mkdir /content/mmsegmentation/data
!mkdir /content/mmsegmentation/data/ade

In [7]:
!unzip -q /content/ade20k/ADEChallengeData2016.zip -d /content/mmsegmentation/data/ade

In [8]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.12.1+cu113 True
0.27.0


In [9]:
!mkdir checkpoints

!wget https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_512x512_160k_ade20k/segformer_mit-b5_512x512_160k_ade20k_20210726_145235-94cedf59.pth -P checkpoints

--2022-08-16 12:28:32--  https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_512x512_160k_ade20k/segformer_mit-b5_512x512_160k_ade20k_20210726_145235-94cedf59.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.89.140.71
Connecting to download.openmmlab.com (download.openmmlab.com)|47.89.140.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 328404259 (313M) [application/octet-stream]
Saving to: ‘checkpoints/segformer_mit-b5_512x512_160k_ade20k_20210726_145235-94cedf59.pth’

segformer_mit-b5_51 100%[===================>] 313.19M  7.27MB/s    in 41s     

2022-08-16 12:29:14 (7.66 MB/s) - ‘checkpoints/segformer_mit-b5_512x512_160k_ade20k_20210726_145235-94cedf59.pth’ saved [328404259/328404259]



In [10]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [12]:

from mmcv import Config

config_file='/content/mmsegmentation/configs/segformer/test_segformer_mit-b5_512x512_160k_ade20k.py'

checkpoint_file = 'checkpoints/segformer_mit-b5_512x512_160k_ade20k_20210726_145235-94cedf59.pth'

cfg = Config.fromfile(config_file)


In [13]:
'''
from mmseg.apis import set_random_seed
from mmseg.utils import get_device


# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg

cfg.runner.max_iters = 200
cfg.log_config.interval = 10
cfg.evaluation.interval = 200
cfg.checkpoint_config.interval = 200

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = checkpoint_file

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'



# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = get_device()

'''

"\nfrom mmseg.apis import set_random_seed\nfrom mmseg.utils import get_device\n\n\n# Since we use only one GPU, BN is used instead of SyncBN\ncfg.norm_cfg = dict(type='BN', requires_grad=True)\ncfg.model.backbone.norm_cfg = cfg.norm_cfg\ncfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg\n\ncfg.runner.max_iters = 200\ncfg.log_config.interval = 10\ncfg.evaluation.interval = 200\ncfg.checkpoint_config.interval = 200\n\n# We can still use the pre-trained Mask RCNN model though we do not need to\n# use the mask branch\ncfg.load_from = checkpoint_file\n\n# Set up working dir to save files and logs.\ncfg.work_dir = './work_dirs/tutorial'\n\n\n\n# Set seed to facitate reproducing the result\ncfg.seed = 0\nset_random_seed(0, deterministic=False)\ncfg.gpu_ids = range(1)\ncfg.device = get_device()\n\n"

In [14]:
from mmseg.apis import set_random_seed
from mmseg.utils import get_device

cfg.load_from = checkpoint_file
cfg.work_dir = './work_dirs/tutorial'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = get_device()


In [15]:
# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='BN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained=
    'https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/segformer/mit_b5_20220624-658746d9.pth',
    backbone=dict(
        type='MixVisionTransformer',
        in_channels=3,
        embed_dims=64,
        num_stages=4,
        num_layers=[3, 6, 40, 3],
        num_heads=[1, 2, 5, 8],
        patch_sizes=[7, 3, 3, 3],
        sr_ratios=[8, 4, 2, 1],
        out_indices=(0, 1, 2, 3),
        mlp_ratio=4,
        qkv_bias=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.1),
    decode_head=dict(
        type='SegformerHead',
        in_channels=[64, 128, 320, 512],
        in_index=[0, 1, 2, 3],
        channels=256,
        dropout_ratio=0.1,
        num_classes=150,
        norm_cfg=dict(type='BN', requires_grad=True),
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_

In [16]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
import mmcv
import os.path as osp


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(cfg.model)

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

2022-08-16 12:33:20,892 - mmseg - INFO - Loaded 20210 images
/content/mmsegmentation/mmseg/models/backbones/mit.py:365: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '
2022-08-16 12:33:28,148 - mmseg - INFO - Loaded 2000 images
2022-08-16 12:33:28,151 - mmseg - INFO - load checkpoint from local path: checkpoints/segformer_mit-b5_512x512_160k_ade20k_20210726_145235-94cedf59.pth
2022-08-16 12:33:28,623 - mmseg - INFO - Start running, host: root@5f8195607667, work_dir: /content/mmsegmentation/work_dirs/tutorial
2022-08-16 12:33:28,625 -

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2000/2000, 6.5 task/s, elapsed: 309s, ETA:     0s

2022-08-16 12:41:32,926 - mmseg - INFO - per class results:
2022-08-16 12:41:32,941 - mmseg - INFO - 
+---------------------+-------+-------+
|        Class        |  IoU  |  Acc  |
+---------------------+-------+-------+
|         wall        | 77.11 | 88.32 |
|       building      | 81.81 | 91.55 |
|         sky         | 93.97 | 96.85 |
|        floor        | 81.22 | 90.45 |
|         tree        | 73.49 | 87.67 |
|       ceiling       | 84.53 | 92.99 |
|         road        | 83.29 |  91.0 |
|         bed         |  88.8 | 94.65 |
|      windowpane     | 59.94 | 75.72 |
|        grass        | 68.09 |  84.3 |
|       cabinet       | 60.73 | 73.35 |
|       sidewalk      | 65.13 | 80.56 |
|        person       | 79.74 | 91.01 |
|        earth        |  37.9 | 52.61 |
|         door        | 46.98 | 59.39 |
|        table        |  59.1 | 74.69 |
|       mountain      | 58.53 | 72.24 |
|        plant        | 51.13 | 64.46 |
|       curtain       | 72.49 | 84.13 |
|        chair    

In [18]:
import matplotlib.pyplot as plt
from mmseg.core.evaluation import get_palette
model.cfg = cfg
#img = '/content/mmsegmentation/data/ade/ADEChallengeData2016/images/validation/ADE_val_00001000.jpg'
resimler=['validation/ADE_val_00001962.jpg' , 'validation/ADE_val_00001963.jpg' , 'validation/ADE_val_00001457.jpg' , 
          'validation/ADE_val_00001458.jpg' , 'validation/ADE_val_00001459.jpg' , 'training/ADE_train_00000565.jpg' ,
          'training/ADE_train_00000936.jpg' , 'training/ADE_train_00000952.jpg' , 'training/ADE_train_00003194.jpg' , 
          'training/ADE_train_00006845.jpg']
for img in resimler:
  result = inference_segmentor(model, '/content/mmsegmentation/data/ade/ADEChallengeData2016/images/'+img)
  show_result_pyplot(model, '/content/mmsegmentation/data/ade/ADEChallengeData2016/images/'+img, result,get_palette('ade20k'))

Output hidden; open in https://colab.research.google.com to view.